# Intro to Biomedical Ontologies: Owlready2

Biomedical ontologies is generally a tough-to-approach field, starting with "what is an ontology?".

I often reply to that with "it's a hairball of knowledge." 

Imagine if someone/group decided to "lets represent something close to a neuronal-connection of knowledge".

For me, I am not an ontologist (creating new ontologies). I consider myself one of the few people who can figure out how to leverage ontologies to achieve very specific biomedical and clinical tasks. 

In [1]:
import owlready2

hpo = owlready2.get_ontology("http://purl.obolibrary.org/obo/hp.owl").load()
# mondo = owlready2.get_ontology("http://purl.obolibrary.org/obo/mondo.owl").load()
# efo = owlready2.get_ontology("http://www.ebi.ac.uk/efo/efo.owl").load()


In [100]:
search_term = "moyamoya"

# Crude searcher
def obo_searcher(ontology, search_term):
    mondo_results = ontology.search(label = f"*{search_term}*", _case_sensitive=False)
    data = [{
        "concept": x,
        "label": x.label,
        "iri": x.iri,
        "synonyms": x.hasExactSynonym,
        "name": x.name,
        "subclasses": list(x.subclasses()),
        "xrefs": x.hasDbXref
    } for x in mondo_results if str(x.label)]

    return (data)


In [101]:
results = obo_searcher(hpo, search_term)

# Results: List of dictionary of ontology concepts and metadata

In [102]:
results

[{'concept': obo.HP_0011834,
  'label': ['Moyamoya phenomenon'],
  'iri': 'http://purl.obolibrary.org/obo/HP_0011834',
  'synonyms': [],
  'name': 'HP_0011834',
  'subclasses': [],
  'xrefs': ['UMLS:C4023169']}]

## Check one concept

In [103]:
results

[{'concept': obo.HP_0011834,
  'label': ['Moyamoya phenomenon'],
  'iri': 'http://purl.obolibrary.org/obo/HP_0011834',
  'synonyms': [],
  'name': 'HP_0011834',
  'subclasses': [],
  'xrefs': ['UMLS:C4023169']}]

In [104]:
results[0]['concept'].hasExactSynonym

[]

## Get `is_a` concepts

In [105]:
is_a = results[0]['concept'].is_a

# it returns a list, since "stomach cancer" can be multiple things
[x.label for x in is_a]

[['Abnormal cerebral artery morphology']]

## Get all ancestors

In [106]:
ancestors = results[0]['concept'].ancestors()

# it returns a list of all ancestors
[x.label for x in ancestors]

[['Abnormality of cardiovascular system morphology'],
 ['Morphological central nervous system abnormality'],
 ['Abnormality of brain morphology'],
 ['Abnormal blood vessel morphology'],
 ['Abnormal cerebral artery morphology'],
 ['Abnormal systemic arterial morphology'],
 ['Abnormality of the cardiovascular system'],
 ['Abnormal cerebral vascular morphology'],
 ['Abnormality of the nervous system'],
 [],
 ['Phenotypic abnormality'],
 ['Moyamoya phenomenon'],
 ['Abnormal nervous system morphology'],
 ['All'],
 [locstr('Abnormal vascular morphology', 'en')],
 ['Abnormality of the vasculature']]

## Get all descendants

In [107]:
descendants = results[0]['concept'].descendants()

# Descendants tends to return itself
[(x.label, x.name) for x in descendants]

[(['Moyamoya phenomenon'], 'HP_0011834')]

## Get all Subclasses

In [108]:
subclasses = results[0]['concept'].subclasses()

# Descendants tends to return itself
[(x.label, x.name) for x in subclasses]

[]

## Things to Note:

- `label`: actually returns a list of the synonyms related
- `iri`: unique ID for this concept
- `name`: concept ID, Even though this is an HPO term, sometimes ontologies can reference external ontologies as part of the "semantic web" reference.
- `xrefs`: Generally, `owlready2` has poor documentation, but it's a single person(?) effort (and I never personally contributed) for not the most approachable field, so give him some slack. But the oddly named `.hasDbXref` returns a list of external cross-walks, which is one of the more useful things to know.

In [109]:
def serialize_obo_subclass(item):
    subclasses = item['subclasses'] 
    if len(subclasses) == 0:
        return item
    else: 
        serialized = [{
            "label": subclass.label,
            "iri": subclass.iri,
            "subclasses": [x.name for x in list(subclass.subclasses())],
            "xrefs": subclass.hasDbXref,
        } for subclass in subclasses]
        return serialized

In [110]:
# serialized_result = [serialize_obo_subclass(item) for item in items]

# TODO Visual Interface

In [111]:
import altair as alt


search_input = alt.param(
    value='',
    bind=alt.binding(
        input='search',
        placeholder="Diseases/symtoms",
        name='Search ',
    )
)
alt.Chart().mark_point(size=60).encode(
    x='x:Q',
    y='y:Q',
    # tooltip='Name:N',
    # opacity=alt.condition(
    #     alt.expr.test(alt.expr.regexp(search_input, 'i'), alt.datum.all_labels),
    #     alt.value(1),
    #     alt.value(0.05)
    # )
).add_params(
    search_input
)

alt.Chart(...)